In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:

for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

46 documents
12 classes ['Experience', 'Projects', 'Volunteering', 'awards', 'bio', 'certificates', 'education', 'greeting', 'name', 'school', 'skill', 'thanks']
57 unique lemmatized words ["'s", ',', 'about', 'and', 'anyone', 'are', 'award', 'awesome', 'bio', 'can', 'certificate', 'certification', 'college', 'day', 'done', 'education', 'ekse', 'experience', 'for', 'gained', 'good', 'have', 'hello', 'helpful', 'helping', 'hey', 'hi', 'him', 'his', 'hola', 'how', 'in', 'internship', 'is', 'linkedin', 'me', 'name', 'ny', 'of', 'project', 'school', 'skill', 'soumyo', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'volunteering', 'what', 'whats', 'who', 'work', 'you', 'your', 'yourself']


In [ ]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [ ]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [ ]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-8-9ffac2c3f218>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
train_x = np.array(train_x)
train_y = np.array(train_y)

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

model.save('model.h5', hist)

print("model created")

Epoch 1/200


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


10/10 [==============================] - 1s 5ms/step - loss: 2.4866 - accuracy: 0.0435
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.4294 - accuracy: 0.2391
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.3631 - accuracy: 0.2609
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 2.2906 - accuracy: 0.3261
Epoch 5/200
10/10 [==============================] - 0s 3ms/step - loss: 2.2129 - accuracy: 0.3696
Epoch 6/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1128 - accuracy: 0.3696
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 1.9780 - accuracy: 0.4348
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 1.8174 - accuracy: 0.4565
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 1.6376 - accuracy: 0.6304
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 1.4295 - accuracy: 0.5870
Epoch 11/200
10/10 [=